In [0]:
!pip3 install spacy_udpipe
!pip3 install wiki_ru_wordnet
!pip3 install pymorphy2
!pip3 install yargy

In [0]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import dictionary, gte, lte, gram, custom
from yargy.pipelines import morph_pipeline

In [0]:
from wiki_ru_wordnet import WikiWordnet
from collections import defaultdict
from google.colab import files
import spacy_udpipe
import ufal.udpipe
import pymorphy2
import string
import re

morph = pymorphy2.MorphAnalyzer()
wikiwordnet = WikiWordnet()

In [8]:
spacy_udpipe.download("ru")
nlp = spacy_udpipe.load("ru")

Successfully downloaded the pretrained UDPipe model for the 'ru' language


In [0]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [11]:
uploaded = files.upload()

Saving decript_clean.txt to decript_clean.txt


In [0]:
texts = uploaded['decript_clean.txt'].decode('UTF-8')
texts = texts.split('\n')

In [0]:
opn_dates_log = uploaded['log_open_dates.txt'].decode('UTF-8')
opn_dates_log = opn_dates_log.split('\n')

**Алгоритм**

1. Проходимся по текстам описаний
2. Каждый делим на предложения 
3. Ищем дату
4. Если есть --> находим глагол
5. Если глагол в словаре --> смотрим на зависимые
6. Если в зависимых библиотека - отлично 
7. Если нет --> ищем с помощью yargy сочетания СУЩ В ИМ + библиотека 
8. Если сущ в зависимых - отлично 


1.   Дата открытия библиотеки – 1916 год +
2.   Дата основания – +
3.   Год открытия – 2001. +
4.   Образована в 1964 г. +
5.   Дата основания: конец 1919 г. – начало 1920 г. +
6. берет свое начало -
7. Официальная дата рождения областной специальной библиотеки для слепых – 5 сентября 1955 года. +
8. Центральная Межпоселенческая библиотека – самая крупная библиотека Сосновского района, дата создания – 1935 год. +



In [0]:
Date = fact('Date', ['day', 'month', 'year'])

MONTHS = {
    'январь', 'февраль', 'март',
    'апрель', 'мая', 'июнь',
    'июль', 'август', 'сентябрь',
    'октябрь', 'ноябрь', 'декабрь'}


MONTH_NAME = dictionary(MONTHS)
DAY = and_(gte(1), lte(31))
YEAR = and_(gte(1900), lte(2100))


a = rule(
    dictionary({'дата', 'год'}),
    gram('NPRO').optional(), 
    dictionary({'основания', 'открытия', 'рождения', 'создания', 'образования'})
)


b = rule(
    gram('ADJF').optional().repeatable(),
    dictionary({'библиотеки'}).optional(),
    gram('PREP').optional(),
    or_(gram('NOUN'), gram('ADJF')).optional()
)

c = dictionary({'–', ':'})
d = dictionary({'конец', 'середина', 'начало'}).optional()


DATE1 = rule(
    
    DAY.interpretation(
        Date.day).optional(),
    
    MONTH_NAME.interpretation(
        Date.month).optional(),
    
    YEAR.interpretation(Date.year)
    
).interpretation(Date)


# Если объединить, получится не очень 

DATE2 = rule(
    
    a, b, c, d,
    
    DAY.interpretation(Date.day).optional(),
    
    MONTH_NAME.interpretation(
        Date.month.normalized()).optional(),
    
    YEAR.interpretation(Date.year)
           
    ).interpretation(Date)


open_date_parser1 = Parser(DATE1)
open_date_parser2 = Parser(DATE2)

In [0]:
open_date_verb = [
                  'возникла', 'возобновилась', 'доступна',
                  'началась', 'образована',  'организованна',
                  'организована', 'основана', 'открылась', 
                  'открыта', 'появилась', 'учреждена',
                  'создана', 
]

In [0]:
texts = ['Левковская сельская библиотека основана в 1968 году.', 
         'Петровская сельская библиотека была открыта в 1931 году.', 
         'Центральная детская библиотека была открыта 1 июля 1950 года.',
         'В 1974 году открылась библиотека.', 
         'Углевская сельская библиотека основана в 1897 году.', 
         'Заведующая библиотекой – Людмила Олеговна Веткина.26 марта 1966 года открыта библиотека-филиал № 10.',
         'Библиотека работает с 1935 года.']

In [0]:
length = len(texts)

In [0]:
open_date = []
log = []
count = 0

out = display(progress(0, length), display_id=True)

for index, text in enumerate(texts):
  doc = nlp(text)

  ans = ''

  for sent in doc.sents:

    match = list(open_date_parser1.findall(sent.text))
      
    if match != [] and ans == '':
      mat = list(open_date_parser2.findall(sent.text))
      
      # берем первое вхождение, потому что дата открытия скорее будет в начале текста

      if mat != []:
        ans = ' '.join([i for i in mat[0].fact if i is not None])
        count += 1
      elif str(sent.root).lower() in open_date_verb:
        ans = ' '.join([i for i in match[0].fact if i is not None])
        count += 1
   
  if ans == '':
    log.append(index)

  open_date.append(ans)
  out.update(progress(index, length))
  
      

In [0]:
count

3801

In [0]:
Часто встречается "изба-читальня"

In [0]:
with open('open_dates.txt', 'w', encoding='utf-8') as f:
  for i in open_date:
    f.write(i + '\n')

In [0]:
files.download('submission111111.csv')

In [0]:
files.download('open_dates.txt') 

In [0]:
with open('log_open_dates.txt', 'w', encoding='utf-8') as f:
  for i in log:
    f.write(str(i) + '\n')

files.download('log_open_dates.txt') 

###  Брать начало



*   Библиотека села Чувичи берет свое начало от избы-читальни, основанной в 1932 году в школе.
*  Ее «биография» берет свое начало в 1952 году
*  Сасовская центральная библиотека берет свое начало с 1919 года
*  История библиотеки берет свое начало с 1918 года.
*  История библиотеки «Книгочей» берет свое начало в 40-х годах ХХ века
* История Янаульской центральной районной библиотеки берет свое начало в далеком 1934 году.
* История создания библиотеки берет свое начало со второй половины XX века.




*   ХІХ–ХХ веков
*   20-х годах 20 века
*  в первой половине XX века
*  в 20-е годы прошлого века
* В начале XX века



In [0]:
Date = fact('Date', 
            ['decade', 'half', 'part', 'time', 'century'])

Date = fact('Date', 
            ['decade'])

HALF = dictionary({'первый', 'второй'})
PART1 = dictionary({'начало', 'середина', 'конец', 'половина'})
TIME = dictionary({'прошлый', 'позапрошлый'})
CENTURY = dictionary({'XX', 'XIX'})
DECADE = and_(gte(10), lte(90))



DATE1 = rule(
    
    DECADE.interpretation(
        Date.decade),

).interpretation(Date)

open_date_parser1 = Parser(DATE1)

In [0]:
txt = ['20-х годах 20 века', 'в 20-е годы прошлого века', 'В начале XX века', 
       'в 20-е годы прошлого века', 'в первой половине XX века', 'ХІХ–ХХ веков']

for i in txt:
  match = open_date_parser1.findall(i)

  for l in match:
    print(l.fact)

In [0]:
from yargy.tokenizer import MorphTokenizer

In [0]:
tokenizer = MorphTokenizer()

In [0]:
for i in tokenizer('20-х годах 20 века'):
  print(i)

Token('20', [0, 2), 'INT')
Token('-', [2, 3), 'PUNCT')
MorphToken('х', [3, 4), 'RU', [Form('х', Grams(UNKN))])
MorphToken('годах', [5, 10), 'RU', [Form('год', Grams(NOUN,inan,loct,masc,plur))])
Token('20', [11, 13), 'INT')
MorphToken('века', [14, 18), 'RU', [Form('век', Grams(NOUN,gent,inan,masc,sing)), Form('веко', Grams(NOUN,gent,inan,neut,sing)), Form('век', Grams(NOUN,inan,masc,nomn,plur)), Form('век', Grams(NOUN,accs,inan,masc,plur))])


**ДОДЕЛАТЬ**

## **Избы-читальни и Библиорум**

библиоroom - это пространство альтернативного времяпрепровождения для активных и креативных молодых людей, предпочитающих живое общение. Учение - свет. Буква - это место, где всегда светло!

Чем же Вы можете заняться в нашем антикафе… Да чем пожелаете! А мы лишь предлагаем варианты:
- Поиграть в настольные игры
- Посидеть в интернете - WiFi у нас бесплатный!
- Познакомиться с новыми интересными людьми;
- Посмотреть интересный фильм;
- Почитать умные книжки и обсудить их за чашечкой чая/кофе:)
Все услуги предоставляются бесплатно.

* изба-читальня
* библиоroom
* библиорум
* библиотеки-читальни

Два случая:


*   Изба-читальня была в городе, библиотека появилась независимо
*   Библиотека появилась на основе избы-читальни



**Test**

In [0]:
RULE = morph_pipeline([
    'изба-читальня', 
    'библиотека-читальня',
    'библиорум', 
    'библиоroom',
])


In [39]:
txt = ['Шаймуратовская сельская библиотека основана в 1953 году (изба-читальня). Мама папа', 
       'Изба-читальня находилась в здании сельского совета. ', 
       'В 1905 году на общественных началах была открыта библиотека-читальня.', 
       'Библиотека основана в 1940 году как изба-читальня.',
       'Библиотека основана в 1940 году как избу-читальню. Изба-читальня.',
       'Мама папа. Библиотека основана в 1940 году как библиорум.']

parser = Parser(RULE)

for text in txt:

  for match in parser.findall(text):
    print([_.value for _ in match.tokens])

  print()

['изба', '-', 'читальня']

['Изба', '-', 'читальня']

['библиотека', '-', 'читальня']

['изба', '-', 'читальня']

['избу', '-', 'читальню']
['Изба', '-', 'читальня']

['библиорум']



**Search**

In [0]:

IZBA = rule(
    rule(dictionary('как')).optional(),
    rule(
   morph_pipeline([
    'изба-читальня', 
    'библиотека-читальня',
    'библиорум', 
    'библиоroom',
])))

izba_parser = Parser(IZBA)

Система разметки: 


*   1 - в тексте встретилось "как изба-читальня"
*   0 - в тексте встретилось "изба-читальня"
*   "" - ничего не встретилось




In [41]:
izba = []
out = display(progress(0, length), display_id=True)

for index, text in enumerate(texts):

  ans = list(izba_parser.findall(text))

  if ans:
    toks = [_.value for _ in ans[0].tokens]
    if toks[0] == 'как': izba.append(1)
    else: izba.append(0)
  else: izba.append('')

  out.update(progress(index, length))

In [0]:
with open('izba.txt', 'w', encoding='utf-8') as f:
  for i in izba:
    f.write(str(i) + '\n')

In [0]:
files.download('submission111111.csv')

In [0]:
files.download('izba.txt') 